<a href="https://colab.research.google.com/github/akbarjimi/BlossomNet/blob/main/Iris_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Load Dataset 🔄

In [19]:
import os
import numpy as np
import random
import csv
from typing import List, Tuple, Any
import math
import pickle

In [20]:
def load_dataset(file_path: str) -> Tuple[List[List[float]], List[Any]]:
    """
    Loads the dataset from a CSV file.

    This function assumes that the CSV file has a header row and that:
    - All columns except the last one are features (converted to float).
    - The last column is the label (left as a string; convert if needed).

    Note:
      The current implementation is particularly suited for datasets like the Iris dataset.
      For other datasets, you might want to modify the logic (e.g., to change the label column index).

    Parameters:
      file_path (str): Path to the dataset file.

    Returns:
      Tuple[List[List[float]], List[Any]]:
          - data: A list of rows, each row is a list of features as floats.
          - labels: A list of labels corresponding to each row.
    """
    data: List[List[float]] = []
    labels: List[Any] = []

    try:
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            header = next(reader, None)  # Skip header row if exists

            # Process each row in the CSV file
            for row in reader:
                if row:  # Ensure the row is not empty
                    # Convert all columns except the last one into floats (features)
                    try:
                        features = [float(item) for item in row[:-1]]
                    except ValueError as ve:
                        print(f"Could not convert features to float in row: {row}. Error: {ve}")
                        continue
                    # The last column is considered the label (remains as string or processed further)
                    label = row[-1]

                    data.append(features)
                    labels.append(label)

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

    return data, labels

In [21]:
    dataset, labels = load_dataset('/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv')
    print(dataset[:3],"\n")
    print(labels[:3])

[[1.0, 5.1, 3.5, 1.4, 0.2], [2.0, 4.9, 3.0, 1.4, 0.2], [3.0, 4.7, 3.2, 1.3, 0.2]] 

['Iris-setosa', 'Iris-setosa', 'Iris-setosa']


In [22]:
def validate_dataset(dataset: List[List[Any]]) -> bool:
    """
    بررسی صحت ساختار و یکپارچگی مجموعه داده.

    این تابع بررسی می‌کند که:
      - مجموعه داده خالی نباشد.
      - تمامی ردیف‌های مجموعه داده دارای تعداد ویژگی یکسان باشند.

    پارامترها:
      dataset (List[List[Any]]): لیستی از ردیف‌های داده؛ هر ردیف نیز لیستی از ویژگی‌هاست.

    خروجی:
      bool: اگر مجموعه داده صحیح باشد، مقدار True و در غیر این صورت False برمی‌گرداند.
    """
    # اگر مجموعه داده خالی باشد، معتبر نیست.
    if not dataset:
        return False

    # تعداد ویژگی‌ها در اولین ردیف را به عنوان مبنا در نظر می‌گیریم.
    num_features = len(dataset[0])
    for row in dataset:
        # بررسی می‌کند که هر ردیف همان تعداد ویژگی با ردیف اولیه داشته باشد.
        if len(row) != num_features:
            return False
    return True

In [23]:
    # نمونه‌ای از یک مجموعه داده معتبر (مثلاً برای دیتاست Iris یا سایر دیتاست‌ها)
    valid_dataset = [
        [5.1, 3.5, 1.4, 0.2],
        [4.9, 3.0, 1.4, 0.2],
        [6.2, 3.4, 5.4, 2.3]
    ]

    # نمونه‌ای از یک مجموعه داده نامعتبر (یک ردیف با تعداد ویژگی متفاوت)
    invalid_dataset = [
        [5.1, 3.5, 1.4, 0.2],
        [4.9, 3.0, 1.4],  # این ردیف تعداد ویژگی کمتری نسبت به بقیه دارد.
        [6.2, 3.4, 5.4, 2.3]
    ]

    # اجرای تابع و چاپ نتایج
    print("مجموعه داده معتبر:", validate_dataset(valid_dataset))    # خروجی: True
    print("مجموعه داده نامعتبر:", validate_dataset(invalid_dataset))  # خروجی: False

مجموعه داده معتبر: True
مجموعه داده نامعتبر: False


In [24]:
def test_data_loading(file_path: str) -> None:
    """
    آزمون عملکرد تابع بارگذاری داده.

    ورودی:
      file_path (str): مسیر فایل CSV برای بارگذاری داده‌ها.

    خروجی:
      None: نتیجه تست از طریق چاپ پیام به کنسول نمایش داده می‌شود.

    توضیحات:
      این تابع ابتدا داده‌ها و برچسب‌ها را از فایل ورودی با استفاده از load_dataset می‌خواند.
      سپس با استفاده از validate_dataset اعتبار داده‌ها را بررسی می‌کند.
      در صورت موفقیت، پیام موفقیت را چاپ می‌کند و در صورت بروز خطا، پیغام مناسب را چاپ می‌کند.
    """
    try:
        dataset, labels = load_dataset(file_path)
        assert validate_dataset(dataset), "Dataset validation failed."
        print("Data loaded and validated successfully.")
    except AssertionError as error:
        print(f"Test failed: {error}")
    except FileNotFoundError:
        print("File not found. Make sure the dataset file exists at the specified path.")
    except Exception as error:
        print(f"An error occurred: {error}")

In [25]:
test_data_loading('/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv')

Data loaded and validated successfully.


# 2. Normalize the Data 🔄

In [26]:
def transpose(data: List[List]) -> List[List]:
    """
    ترانهاده کردن یک لیست دوبعدی.

    پارامترها:
        data (List[List]): داده‌ها به‌صورت لیست از لیست‌ها (مثلاً ماتریس)

    خروجی:
        List[List]: داده‌های ترانهاده‌شده
    """
    # علامت ستاره داده ها را آنپک میکند
    # تابع زیپ ستون n از هر سطر را کنار هم قرار میدهد و بدین وسیله داده ها را ترانسپوزه میکند.
    return [list(row) for row in zip(*data)]

In [27]:
def min_max_normalizer(data: List[List[float]]) -> List[List[float]]:
    """
    این تابع داده‌های عددی رو با استفاده از روش Min-Max نرمال می‌کنه.
    یعنی همه‌ی اعداد رو بین 0 و 1 میاره، تا مقایسه و آموزش مدل ساده‌تر بشه.
    """

    # چرخوندن ماتریس برای بررسی هر ویژگی به‌صورت ستونی
    transposed_data = transpose(data)

    # پیدا کردن مینیمم و ماکزیمم هر ستون
    min_vals = [min(col) for col in transposed_data]
    max_vals = [max(col) for col in transposed_data]

    scaled_data = []

    # نرمال کردن هر مقدار با استفاده از فرمول min-max
    for row_index, row in enumerate(data):
        scaled_row = []
        for val, min_val, max_val in zip(row, min_vals, max_vals):
            range_val = max_val - min_val
            if range_val == 0:
                scaled_row.append(0.0)  # اگر همه مقادیر برابر بودن، خروجی رو صفر بزار
            else:
                scaled_val = (val - min_val) / range_val
                scaled_row.append(scaled_val)
        scaled_data.append(scaled_row)

    return scaled_data


In [28]:
def split_data(dataset: List[List[float]], training_size: float = 0.7, validation_size: float = 0.15) -> Tuple[List[List[float]], List[List[float]], List[List[float]]]:
    """
    این تابع دیتاست رو به سه بخش تقسیم می‌کنه: آموزش، اعتبارسنجی و تست.

    پارامترها:
    dataset (list of lists): دیتای ورودی که می‌خوایم تقسیمش کنیم.
    training_size (float): درصدی از دیتا که برای آموزش استفاده میشه (مثلاً ۰.۷ یعنی ۷۰٪).
    validation_size (float): درصدی از دیتا که برای اعتبارسنجی استفاده میشه (مثلاً ۰.۱۵ یعنی ۱۵٪).

    خروجی:
    یه تاپل شامل سه تا لیسته: دیتای آموزش، دیتای اعتبارسنجی، دیتای تست.
    """

    # اول دیتا رو به صورت تصادفی قاطی می‌کنیم که ترتیبش تأثیر نذاره
    random.shuffle(dataset)

    # اندازه کل دیتا
    total_size = len(dataset)

    # محاسبه مرز بین بخش‌ها
    train_end = int(training_size * total_size)
    val_end = int((training_size + validation_size) * total_size)

    # برش دادن دیتا بر اساس درصدها
    training_data = dataset[:train_end]
    validation_data = dataset[train_end:val_end]
    test_data = dataset[val_end:]

    return training_data, validation_data, test_data


In [29]:
data = [
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
    [6.2, 3.4, 5.4, 2.3],
    [5.9, 3.0, 5.1, 1.8],
    [5.4, 3.9, 1.7, 0.4],
    [6.7, 3.1, 4.7, 1.5],
    [5.6, 2.8, 4.9, 2.0],
    [5.7, 2.1, 4.0, 2.1],
    [5.8, 2.2, 9.4, 1.2],
    [5.9, 2.3, 5.5, 0.8],
]

train, val, test = split_data(data, training_size=0.5, validation_size=0.2)
print("Train:", train)
print("Validation:", val)
print("Test:", test)


Train: [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [6.2, 3.4, 5.4, 2.3], [5.7, 2.1, 4.0, 2.1], [5.6, 2.8, 4.9, 2.0]]
Validation: [[5.9, 2.3, 5.5, 0.8], [5.9, 3.0, 5.1, 1.8]]
Test: [[5.4, 3.9, 1.7, 0.4], [5.8, 2.2, 9.4, 1.2], [6.7, 3.1, 4.7, 1.5]]


In [31]:
def test_split_data():
    """
    این تابع بررسی می‌کنه که تابع split_data درست کار می‌کنه یا نه.
    توی تست، اول دیتاست رو لود می‌کنیم، بعد نرمالایزش می‌کنیم، بعد تقسیمش می‌کنیم.
    بعدش چک می‌کنیم که هر بخش از دیتا خالی نباشه و مجموعشون هم برابر با دیتای اولیه باشه.
    """
    try:
        # مسیر فایل دیتاست
        file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'

        # لود کردن دیتا و لیبل‌ها
        dataset, labels = load_dataset(file_path)

        # نرمال‌سازی دیتا با Min-Max
        dataset = min_max_normalizer(dataset)

        # تقسیم دیتا به سه بخش
        training_data, validation_data, test_data = split_data(dataset)

        # تست اینکه هیچ‌کدوم از بخش‌ها خالی نباشه
        assert len(training_data) > 0, "داده‌های آموزش خالیه."
        assert len(validation_data) > 0, "داده‌های اعتبارسنجی خالیه."
        assert len(test_data) > 0, "داده‌های تست خالیه."

        # تست اینکه تعداد کل داده‌ها تغییر نکرده باشه
        total_size = len(training_data) + len(validation_data) + len(test_data)
        assert total_size == len(dataset), "خطا در تقسیم دیتا: تعداد نهایی با اولیه نمی‌خونه."

        print("✅ تست تقسیم دیتا با موفقیت انجام شد.")

    except AssertionError as error:
        print(f"❌ تست شکست خورد: {error}")

    except Exception as error:
        print(f"⚠️ یه خطایی پیش اومد: {error}")


In [32]:
test_split_data()

✅ تست تقسیم دیتا با موفقیت انجام شد.


# 3. Define the Architecture 🏗

In [33]:
class Neuron:
    """کلاس یک نورون ساده که برای محاسبه خروجی، گرادیان و به‌روزرسانی وزن‌ها استفاده می‌شود."""

    def __init__(self, weights: List[float], bias: float = None):
        """
        نورون با وزن‌ها و بایاس داده‌شده مقداردهی اولیه می‌شود.

        آرگومان‌ها:
        weights: لیستی از وزن‌ها برای ورودی‌های نورون.
        bias: مقدار بایاس برای نورون. اگر داده نشود، یک مقدار تصادفی بین -0.1 و 0.1 به آن اختصاص می‌یابد.
        """
        self.weights = weights
        self.bias = bias if bias is not None else random.uniform(-0.1, 0.1)
        self.inputs = []  # ورودی‌های نورون را ذخیره می‌کند
        self.output = 0   # خروجی نورون را ذخیره می‌کند

    def forward(self, inputs: List[float]) -> float:
        """
        خروجی نورون را با استفاده از ورودی‌ها و وزن‌ها محاسبه می‌کند.

        آرگومان‌ها:
        inputs: لیستی از ورودی‌ها به نورون.

        برمی‌گرداند:
        خروجی نورون.
        """
        self.inputs = inputs  # ذخیره ورودی‌ها برای استفاده در مراحل بعدی
        weighted_sum = sum([input_ * weight for input_, weight in zip(inputs, self.weights)])
        self.output = weighted_sum + self.bias  # جمع کردن وزن‌ها و بایاس
        return self.output

    def activation(self, output: float) -> float:
        """
        تابع فعال‌سازی را بر روی خروجی نورون اعمال می‌کند.

        در اینجا، به طور ساده از تابع سیگموید استفاده می‌کنیم.
        """

        # از تابع فعال‌سازی سیگموید استفاده می‌کنیم
        return 1 / (1 + (2.718 ** -output))  # این یعنی سیگموید

    def compute_gradient(self, delta: float) -> List[float]:
        """
        گرادیان برای وزن‌ها را با استفاده از دلتا (سیگنال خطای لایه بعدی) محاسبه می‌کند.

        آرگومان‌ها:
        delta: سیگنال خطا از لایه بعدی.

        برمی‌گرداند:
        لیستی از گرادیان‌ها برای هر وزن.
        """
        gradients = [delta * input_ for input_ in self.inputs]  # محاسبه گرادیان‌ها
        return gradients

    def update_weights(self, learning_rate: float, gradients: List[float]):
        """
        وزن‌ها و بایاس نورون را با استفاده از گرادیان‌ها و نرخ یادگیری به‌روزرسانی می‌کند.

        آرگومان‌ها:
        learning_rate: نرخ یادگیری برای به‌روزرسانی وزن‌ها.
        gradients: گرادیان‌های محاسبه‌شده برای هر وزن.
        """
        # به‌روزرسانی وزن‌ها
        self.weights = [w - learning_rate * g for w, g in zip(self.weights, gradients)]
        self.bias -= learning_rate * gradients[-1]  # بایاس را نیز به‌روزرسانی می‌کنیم

    def propagate_error_back(self) -> List[float]:
        """
        این متد خطای نورون رو برای لایه قبلی حساب می‌کنه.
        یعنی اول مشتق سیگموید رو از خروجی خودش می‌گیره،
        بعد با هر وزن ضرب می‌کنه تا بگه هر ورودی چقدر در خطا سهم داره.
        """
        # مشتق سیگموید: output * (1 - output)
        deriv = self.output * (1 - self.output)
        # برای هر وزن، سهم خطا = مشتق * وزن
        return [deriv * w for w in self.weights]

In [34]:
# تست عملکرد کلاس نورون

# ایجاد یک نورون با وزن‌ها و بایاس تصادفی
weights = [random.uniform(-1, 1) for _ in range(3)]  # سه ورودی با وزن‌های تصادفی
bias = random.uniform(-0.1, 0.1)  # بایاس تصادفی
neuron = Neuron(weights, bias)

# ورودی‌ها برای نورون
inputs = [0.5, 0.2, 0.8]  # سه ورودی برای نورون

# محاسبه خروجی نورون
output = neuron.forward(inputs)
print(f"خروجی نورون قبل از فعال‌سازی: {output}")

# اعمال تابع فعال‌سازی (سیگموید)
activated_output = neuron.activation(output)
print(f"خروجی نورون بعد از فعال‌سازی: {activated_output}")

# فرض می‌کنیم سیگنال خطا (دلتا) برابر 0.1 است
delta = 0.1
gradients = neuron.compute_gradient(delta)
print(f"گرادیان‌های محاسبه‌شده برای وزن‌ها: {gradients}")

# به‌روزرسانی وزن‌ها با نرخ یادگیری 0.01
neuron.update_weights(learning_rate=0.01, gradients=gradients)
print(f"وزن‌ها و بایاس بعد از به‌روزرسانی: {neuron.weights}, {neuron.bias}")

خروجی نورون قبل از فعال‌سازی: -0.23141342851715974
خروجی نورون بعد از فعال‌سازی: 0.4424093676931212
گرادیان‌های محاسبه‌شده برای وزن‌ها: [0.05, 0.020000000000000004, 0.08000000000000002]
وزن‌ها و بایاس بعد از به‌روزرسانی: [0.35225086873070327, -0.10548841036494608, -0.4501960971073645], -0.028014303123630516


In [37]:
class Layer:
    """یه لایه توی شبکه عصبی که ورودی‌ها رو می‌گیره، می‌فرسته توی نورون‌ها، فعال‌سازی می‌کنه و خروجی می‌سازه."""

    def __init__(self, neurons: List[Neuron], is_output_layer: bool = False):
        """
        آرگومان‌ها:
          neurons: لیستی از نورون‌های این لایه.
          is_output_layer: اگه True باشه، این لایه آخره و خروجی‌شو با softmax می‌سازه.
        """
        self.neurons = neurons
        self.is_output_layer = is_output_layer

    def forward(self, inputs: List[float]) -> List[float]:
        """
        داده‌ها رو می‌فرسته توی هر نورون و خروجی خام هر نورون (logits) رو می‌گیره.
        بعد اگه لایه آخری باشه softmax می‌زنه؛ وگرنه با متد activation خود نورون سیگموید می‌زنه.

        inputs: لیست ورودی به این لایه (مثلاً خروجی لایه قبلی).
        برمی‌گردونه: لیست خروجی نهایی این لایه.
        """
        # logits یعنی «خروجی خام نورون قبل از فعال‌سازی»
        logits = [neuron.forward(inputs) for neuron in self.neurons]

        if self.is_output_layer:
            # پیاده‌سازی دستی softmax: exp هر عدد / مجموع expها
            exp_vals = [math.exp(x) for x in logits]
            sum_exp = sum(exp_vals)
            return [v / sum_exp for v in exp_vals]
        else:
            # برای لایه‌های میانی: هر نورون خودش متد activation داره
            return [neuron.activation(x) for neuron, x in zip(self.neurons, logits)]

    def backward(self, delta: List[float], learning_rate: float) -> List[int]:
        """
        دلتا (خطا) از لایه بعدی رو می‌گیره و برای هر نورون:
        1. با compute_gradient گرادیان‌ها رو محاسبه می‌کنه
        2. با update_weights وزن و بایاس رو آپدیت می‌کنه
        3. با propagate_error_back دلتای مربوط به ورودی‌ها رو دریافت می‌کنه
        در نهایت همه دلتاها رو جمع می‌کنه و برمی‌گردونه برای لایه قبلی.

        delta: لیست خطا برای هر نورون این لایه
        learning_rate: نرخ یادگیری
        برمی‌گردونه: لیست delta برای لایه قبلی
        """
        propagated = []  # این لیست، هر عنصرش لیست خطاهایی‌یه که هر نورون برای ورودی‌ها تولید می‌کنه

        for i, neuron in enumerate(self.neurons):
            # 1. گرادیان رو خود نورون محاسبه می‌کنه
            grads = neuron.compute_gradient(delta[i])

            # 2. خود نورون وزن‌ها و بایاس رو آپدیت می‌کنه
            neuron.update_weights(learning_rate, grads)

            # 3. خطا برای ورودی‌های نورون (برای لایه قبلی)
            propagated.append(neuron.propagate_error_back())

        # جمع کردن دلتاها برای هر ورودی
        # zip(*propagated) ردیف‌ها رو ستونی می‌کنه تا بتونیم جمع کنیم
        prev_delta = [sum(x) for x in zip(*propagated)]
        return prev_delta

In [39]:
neurons = [
    Neuron(weights=[0.5, -0.4], bias=0.1),
    Neuron(weights=[-1.0, 2.0], bias=0.2)
]
layer = Layer(neurons, is_output_layer=False)

layer_output = layer.forward(inputs)
print("خروجی لایه:", layer_output)

delta = [0.1, -0.2]
prev_delta = layer.backward(delta, learning_rate=0.01)
print("دلتا برای لایه قبلی:", prev_delta)

خروجی لایه: [0.5670860322814533, 0.5249766018409759]
دلتا برای لایه قبلی: [0.008541449999999978, 0.10115658000000005]


In [ ]:
class Network:
    """A neural network with multiple layers."""


    def __init__(self, layers: List[Layer], epochs: int, learning_rate: float):
        self.layers = layers
        self.learning_rate = learning_rate
        self.epochs = epochs

    def forward(self, inputs: List[float]) -> List[float]:
        """Propagates inputs through the entire network.

        Args:
            inputs: A list of inputs to the network.

        Returns:
            A list of outputs from the final layer.
        """
        outputs = inputs
        for layer in self.layers:
            outputs = layer.forward(outputs)
        return outputs

    def backward(self, targets: List[float], outputs: List[float]):
        """Performs backpropagation to update the network's weights and biases.

        Args:
            targets: A list of target outputs.
            outputs: A list of outputs from the final forward pass.
        """
        delta = self.loss_derivative(outputs, targets)

        for layer in reversed(self.layers):
            delta = layer.backward(delta, self.learning_rate)

    def compute_loss(self, predicted: List[float], actual: List[float]) -> float:
        """Calculates the loss for the predictions."""
        return LossFunction.cross_entropy(predicted, actual)

    def loss_derivative(self, outputs: List[float], targets: List[float]) -> List[float]:
        """Computes the derivative of the loss function."""
        return [pred - target for pred, target in zip(outputs, targets)]

    def train(self, training_data: List[tuple]):
        """Trains the network on the given data without mini-batches.

        Args:
            training_data: A list of tuples containing input and target data.
        """
        num_samples = len(training_data)

        for epoch in range(self.epochs):
            total_loss = 0
            random.shuffle(training_data)

            for inputs, targets in training_data:
                outputs = self.forward(inputs)

                loss = self.compute_loss(outputs, targets)
                total_loss += loss

                self.backward(targets, outputs)

            avg_loss = total_loss / num_samples

            print(f"Epoch {epoch + 1}/{self.epochs} complete. Average loss: {avg_loss:.4f}")

    def evaluate(self, test_data: List[tuple]) -> float:
        """Evaluates the network on the test data."""
        inputs_batch, targets_batch = zip(*test_data)
        predictions = [self.forward(inputs) for inputs in inputs_batch]
        accuracy = self.calculate_accuracy(predictions, targets_batch)
        return accuracy

    def predict(self, new_data: List[float]) -> List[float]:
        """Predicts the output for new input data.

        Args:
            new_data: A list of new input data.

        Returns:
            A list of predicted outputs.
        """
        return self.forward(new_data)

    def calculate_accuracy(self, predictions: List[List[float]], targets: List[List[float]]) -> float:
        """Calculates the accuracy of the model."""
        correct_predictions = 0
        for pred, target in zip(predictions, targets):
            predicted_class = np.argmax(pred)
            true_class = np.argmax(target)
            if predicted_class == true_class:
                correct_predictions += 1
        return correct_predictions / len(targets)

    def save_weights(self, filename: str):
        """Saves the weights of the network to a file."""
        weights = [[neuron.weights for neuron in layer.neurons] for layer in self.layers]
        biases = [[neuron.bias for neuron in layer.neurons] for layer in self.layers]

        with open(filename, 'wb') as f:
            pickle.dump((weights, biases), f)

    def load_weights(self, filename: str):
        """Loads weights into the network from a file."""
        with open(filename, 'rb') as f:
            weights, biases = pickle.load(f)

        for layer, layer_weights, layer_biases in zip(self.layers, weights, biases):
            for neuron, neuron_weights, neuron_bias in zip(layer.neurons, layer_weights, layer_biases):
                neuron.weights = neuron_weights
                neuron.bias = neuron_bias

In [ ]:
class LossFunction:
    """A utility class for loss functions."""

    @staticmethod
    def cross_entropy(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """Calculates the categorical cross-entropy loss.

        Args:
            predicted_outputs: A list of predicted probabilities for each class.
            actual_outputs: A one-hot encoded list of actual output values.

        Returns:
            The categorical cross-entropy loss.
        """
        # Clip the predicted values to prevent log(0)
        predicted_outputs = np.clip(predicted_outputs, 1e-12, 1 - 1e-12)

        # Calculate the cross-entropy loss
        loss = -sum([actual_output * np.log(predicted_output)
                     for predicted_output, actual_output in zip(predicted_outputs, actual_outputs)])
        return loss / len(predicted_outputs)

    @staticmethod
    def mean_squared_error(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """Calculates the mean squared error (MSE).

        Args:
            predicted_outputs: A list of predicted output values.
            actual_outputs: A list of actual output values.

        Returns:
            The mean squared error.
        """
        squared_errors = [(predicted_output - actual_output) ** 2
                          for predicted_output, actual_output in zip(predicted_outputs, actual_outputs)]
        return sum(squared_errors) / len(squared_errors)


In [ ]:
class WeghitsInitializer:
    """A utility class for weight initialization."""

    @staticmethod
    def weights(num_inputs: int) -> List[float]:
        """Initializes weights with random values uniformly distributed between -0.1 and 0.1.

        Args:
            num_inputs: The number of inputs to the neuron.

        Returns:
            A list of initialized weights.
        """
        return [random.uniform(-0.1, 0.1) for _ in range(num_inputs)]


# 4. Workflow 🔮: Load Dataset

In [ ]:
test_data_loading()
file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'
dataset, labels = load_dataset(file_path)
test_split_data()
training_data, validation_data, test_data = split_data(dataset)
print(training_data)
print(validation_data)
print(test_data)

Data loaded and validated successfully.
Data splitting test passed successfully.
[[38.0, 4.9, 3.1, 1.5, 0.1], [89.0, 5.6, 3.0, 4.1, 1.3], [141.0, 6.7, 3.1, 5.6, 2.4], [4.0, 4.6, 3.1, 1.5, 0.2], [99.0, 5.1, 2.5, 3.0, 1.1], [50.0, 5.0, 3.3, 1.4, 0.2], [111.0, 6.5, 3.2, 5.1, 2.0], [9.0, 4.4, 2.9, 1.4, 0.2], [91.0, 5.5, 2.6, 4.4, 1.2], [123.0, 7.7, 2.8, 6.7, 2.0], [109.0, 6.7, 2.5, 5.8, 1.8], [19.0, 5.7, 3.8, 1.7, 0.3], [24.0, 5.1, 3.3, 1.7, 0.5], [31.0, 4.8, 3.1, 1.6, 0.2], [86.0, 6.0, 3.4, 4.5, 1.6], [132.0, 7.9, 3.8, 6.4, 2.0], [112.0, 6.4, 2.7, 5.3, 1.9], [5.0, 5.0, 3.6, 1.4, 0.2], [15.0, 5.8, 4.0, 1.2, 0.2], [101.0, 6.3, 3.3, 6.0, 2.5], [80.0, 5.7, 2.6, 3.5, 1.0], [57.0, 6.3, 3.3, 4.7, 1.6], [118.0, 7.7, 3.8, 6.7, 2.2], [18.0, 5.1, 3.5, 1.4, 0.3], [126.0, 7.2, 3.2, 6.0, 1.8], [149.0, 6.2, 3.4, 5.4, 2.3], [150.0, 5.9, 3.0, 5.1, 1.8], [27.0, 5.0, 3.4, 1.6, 0.4], [66.0, 6.7, 3.1, 4.4, 1.4], [131.0, 7.4, 2.8, 6.1, 1.9], [114.0, 5.7, 2.5, 5.0, 2.0], [63.0, 6.0, 2.2, 4.0, 1.0], [55.0, 6.5, 

# Workflow 🔮: Architecture

## Create input layer

In [ ]:
weights = WeghitsInitializer.weights(4)
bias = random.uniform(-0.1, 0.1)
input_neurons = [Neuron(weights=weights, bias=bias) for _ in range(4)]
input_layer = Layer(neurons=input_neurons, activation= ActivationFunctions.relu, is_output_layer=False)

# Print weights of input neurons
for neuron in input_layer.neurons:
    print("Input Neuron Weights:", neuron.weights)

Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]
Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]
Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]
Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]


## Create hidden layer

In [ ]:
weights = WeghitsInitializer.weights(4)
bias = random.uniform(-0.1, 0.1)
hidden_neurons = [Neuron(weights=weights, bias=bias) for _ in range(5)]
hidden_layer = Layer(neurons=hidden_neurons, activation= ActivationFunctions.relu, is_output_layer=False)

# Print weights of input neurons
for neuron in hidden_layer.neurons:
    print("Input Neuron Weights:", neuron.weights)

Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]


## Create output layer

In [ ]:
weights = WeghitsInitializer.weights(5)
bias = random.uniform(-0.1, 0.1)
output_neurons = [Neuron(weights=weights, bias=bias) for _ in range(3)]
output_layer = Layer(neurons=output_neurons, activation= ActivationFunctions.relu, is_output_layer=True)

# Print weights of input neurons
for neuron in output_layer.neurons:
    print("Input Neuron Weights:", neuron.weights)

Input Neuron Weights: [0.061455682311839954, -0.013101149463906475, 0.09484125191639023, 0.04079782617445696, -0.049328948326537274]
Input Neuron Weights: [0.061455682311839954, -0.013101149463906475, 0.09484125191639023, 0.04079782617445696, -0.049328948326537274]
Input Neuron Weights: [0.061455682311839954, -0.013101149463906475, 0.09484125191639023, 0.04079782617445696, -0.049328948326537274]


## Train

In [ ]:
# Define network architecture
epochs = 100
learning_rate = 0.01
layers = [input_layer, hidden_layer, output_layer]

# Initialize the network
network = Network(layers=layers, epochs=epochs, learning_rate=learning_rate)

num_samples = len(training_data)
print(num_samples)
print(training_data)
print(labels)

def match_and_combine(upper_list: List[List[float]], lower_list: List[List[float]]) -> List[Tuple[List[float], str]]:
    # Create a dictionary from the lower list
    lower_dict = {item[0]: item[1] for item in lower_list}

    # Create a list to store the result tuples
    result = []

    # Iterate through the upper list
    for item in upper_list:
        key = item[0]
        # Check if the key exists in the lower dictionary
        if key in lower_dict:
            # Create a tuple (without the key) and add to the result list
            result.append((item[1:], lower_dict[key]))

    return result

# Example usage
combined = match_and_combine(training_data, labels)
print(combined)

network.train(combined)


105
[[38.0, 4.9, 3.1, 1.5, 0.1], [89.0, 5.6, 3.0, 4.1, 1.3], [141.0, 6.7, 3.1, 5.6, 2.4], [4.0, 4.6, 3.1, 1.5, 0.2], [99.0, 5.1, 2.5, 3.0, 1.1], [50.0, 5.0, 3.3, 1.4, 0.2], [111.0, 6.5, 3.2, 5.1, 2.0], [9.0, 4.4, 2.9, 1.4, 0.2], [91.0, 5.5, 2.6, 4.4, 1.2], [123.0, 7.7, 2.8, 6.7, 2.0], [109.0, 6.7, 2.5, 5.8, 1.8], [19.0, 5.7, 3.8, 1.7, 0.3], [24.0, 5.1, 3.3, 1.7, 0.5], [31.0, 4.8, 3.1, 1.6, 0.2], [86.0, 6.0, 3.4, 4.5, 1.6], [132.0, 7.9, 3.8, 6.4, 2.0], [112.0, 6.4, 2.7, 5.3, 1.9], [5.0, 5.0, 3.6, 1.4, 0.2], [15.0, 5.8, 4.0, 1.2, 0.2], [101.0, 6.3, 3.3, 6.0, 2.5], [80.0, 5.7, 2.6, 3.5, 1.0], [57.0, 6.3, 3.3, 4.7, 1.6], [118.0, 7.7, 3.8, 6.7, 2.2], [18.0, 5.1, 3.5, 1.4, 0.3], [126.0, 7.2, 3.2, 6.0, 1.8], [149.0, 6.2, 3.4, 5.4, 2.3], [150.0, 5.9, 3.0, 5.1, 1.8], [27.0, 5.0, 3.4, 1.6, 0.4], [66.0, 6.7, 3.1, 4.4, 1.4], [131.0, 7.4, 2.8, 6.1, 1.9], [114.0, 5.7, 2.5, 5.0, 2.0], [63.0, 6.0, 2.2, 4.0, 1.0], [55.0, 6.5, 2.8, 4.6, 1.5], [72.0, 6.1, 2.8, 4.0, 1.3], [3.0, 4.7, 3.2, 1.3, 0.2], [106.0

TypeError: can't multiply sequence by non-int of type 'numpy.float64'